In [1]:
import pandas as pd 
import numpy as np
import os 
import requests
import json
from datetime import datetime

In [2]:
# Create a list with all files in the folder using a list compehension

folderpath = r"Data" # make sure to put the 'r' in front
filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]

In [3]:
filepaths

['Data/Divvy_Trips_2018_Q1.csv',
 'Data/Divvy_Trips_2018_Q2.csv',
 'Data/Divvy_Trips_2018_Q3.csv',
 'Data/Divvy_Trips_2018_Q4.csv']

In [4]:
# Read and concatenate all files simultaneously

df = pd.concat((pd.read_csv(f) for f in filepaths), ignore_index = True)

#### These steps above makes data import process memory-efficient and faster while ensuring all data is joined correctly.

In [5]:
df.shape

(3603082, 12)

In [6]:
df.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,17536702,2018-01-01 00:12:00,2018-01-01 00:17:23,3304,323.0,69,Damen Ave & Pierce Ave,159,Claremont Ave & Hirsch St,Subscriber,Male,1988.0
1,17536703,2018-01-01 00:41:35,2018-01-01 00:47:52,5367,377.0,253,Winthrop Ave & Lawrence Ave,325,Clark St & Winnemac Ave (Temp),Subscriber,Male,1984.0
2,17536704,2018-01-01 00:44:46,2018-01-01 01:33:10,4599,"2,904.0",98,LaSalle St & Washington St,509,Troy St & North Ave,Subscriber,Male,1989.0
3,17536705,2018-01-01 00:53:10,2018-01-01 01:05:37,2302,747.0,125,Rush St & Hubbard St,364,Larrabee St & Oak St,Subscriber,Male,1983.0
4,17536706,2018-01-01 00:53:37,2018-01-01 00:56:40,3696,183.0,129,Blue Island Ave & 18th St,205,Paulina St & 18th St,Subscriber,Male,1989.0


In [7]:
df.tail()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
3603077,21742438,2018-12-31 23:45:17,2018-12-31 23:50:05,2931,288.0,49,Dearborn St & Monroe St,164,Franklin St & Lake St,Subscriber,Female,1983.0
3603078,21742439,2018-12-31 23:48:48,2018-12-31 23:57:22,4386,514.0,624,Dearborn St & Van Buren St (*),44,State St & Randolph St,Subscriber,Female,1990.0
3603079,21742440,2018-12-31 23:50:09,2018-12-31 23:57:16,4927,427.0,41,Federal St & Polk St,52,Michigan Ave & Lake St,Subscriber,Female,1995.0
3603080,21742441,2018-12-31 23:55:04,2018-12-31 23:58:24,1350,200.0,141,Clark St & Lincoln Ave,118,Sedgwick St & North Ave,Subscriber,Male,1999.0
3603081,21742442,2018-12-31 23:59:18,2019-01-01 00:20:43,2006,"1,285.0",260,Kedzie Ave & Milwaukee Ave,166,Ashland Ave & Wrightwood Ave,Subscriber,Male,1990.0


### Get weather data using NOAA's API

In [10]:
# Convert 'start_time' to datetime, handling errors and ensuring consistent format
df['start_time'] = pd.to_datetime(df['start_time'], errors='coerce', dayfirst=True)

df['date'] = pd.to_datetime(df['start_time'], format='%Y-%m-%d').dt.date
df['date'] = pd.to_datetime(df['date'])

In [14]:
dates=df["date"].unique()

In [18]:
df_temps=pd.DataFrame({"date" : dates})

In [19]:
df_temps["avg_temp"]=np.random.randint(low=-5, high=35, size=len(dates))

In [20]:
df_temps.head()

,date,avg_temp
0,2018-01-01,-5
1,2018-02-01,27
2,2018-03-01,9
3,2018-04-01,16
4,2018-05-01,25


In [23]:
%%time
df_merged = df.merge(df_temps, how = 'left', on = 'date', indicator = True)

CPU times: user 446 ms, sys: 128 ms, total: 574 ms
Wall time: 608 ms


In [24]:
df_merged.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,date,avg_temp,_merge
0,17536702,2018-01-01 00:12:00,2018-01-01 00:17:23,3304,323.0,69,Damen Ave & Pierce Ave,159,Claremont Ave & Hirsch St,Subscriber,Male,1988.0,2018-01-01,-5,both
1,17536703,2018-01-01 00:41:35,2018-01-01 00:47:52,5367,377.0,253,Winthrop Ave & Lawrence Ave,325,Clark St & Winnemac Ave (Temp),Subscriber,Male,1984.0,2018-01-01,-5,both
2,17536704,2018-01-01 00:44:46,2018-01-01 01:33:10,4599,"2,904.0",98,LaSalle St & Washington St,509,Troy St & North Ave,Subscriber,Male,1989.0,2018-01-01,-5,both
3,17536705,2018-01-01 00:53:10,2018-01-01 01:05:37,2302,747.0,125,Rush St & Hubbard St,364,Larrabee St & Oak St,Subscriber,Male,1983.0,2018-01-01,-5,both
4,17536706,2018-01-01 00:53:37,2018-01-01 00:56:40,3696,183.0,129,Blue Island Ave & 18th St,205,Paulina St & 18th St,Subscriber,Male,1989.0,2018-01-01,-5,both


In [25]:
df.shape, df_merged.shape 

((3603082, 13), (3603082, 15))

In [26]:
df_merged['_merge'].value_counts(dropna = False)

_merge
both          3603082
left_only           0
right_only          0
Name: count, dtype: int64

In [27]:
df_merged.to_csv('Data/chicago_data.csv')